In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import exposure, filters
from PIL import Image
import skimage.color
import skimage.io
import seaborn as sns
import math

In [ ]:
im = plt.imread('resized_data/example_image_resized/ISIC_0015483.jpg')
plt.imshow(im)

In [ ]:
border=plt.imread('resized_data/example_segmentation_resized/ISIC_0015483_segmentation.png')
plt.imshow(border, cmap='gray')

In [ ]:
#adding the border to the picture

#load the images as objects with Image.open()

borderObj = Image.open('resized_data/example_segmentation_resized/ISIC_0001769_segmentation.png')
imObj = Image.open('resized_data/example_image_resized/ISIC_0001769.jpg')

borderObj.paste(imObj, (0,0), mask = borderObj) 

plt.imshow(borderObj, cmap='gray')

In [ ]:
#greyscale histogram, Color
#image = skimage.io.imread(fname='resized_data/example_image_resized/ISIC_0015372.jpg', as_gray=True)

#show the picture
fig, ax = plt.subplots()
plt.imshow(borderObj, cmap='gray')
plt.show()


#Create the plot with 256 bins (for each range of color)
# histogram, bin_edges = np.histogram(borderObj, bins=255, range=(1, 256))
# fig = plt.figure()
# ax = fig.add_axes((0,0,1,1))
# plt.title("Grayscale Histogram")
# plt.xlabel("grayscale value")
# plt.ylabel("pixel count")
# plt.xlim([0, 256])
# sns.lineplot(bin_edges[1:], histogram, ax=ax)  # <- or here
# plt.show()
# sns.histplot(np.array(borderObj))


# or do it with the colors

#img = im.copy()
#img[border==0] = 0
#plt.imshow(img)
border_arr = np.array(borderObj)
border_arr = border_arr[border_arr != 0]
sns.histplot(border_arr, bins=128)

print(border_arr)

print(border_arr.shape)

In [ ]:
from skimage import morphology

struct_el = morphology.disk(2)

# Use this "brush" to erode the image - eat away at the borders

mask_eroded = morphology.binary_erosion(border, struct_el)

# Show side by side (depending on brush size, you might not see a difference visually)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 3))
axes[0].imshow(border, cmap='gray')
axes[1].imshow(mask_eroded, cmap='gray')
fig.tight_layout()

# Verify the new mask is smaller
new_area= np.sum(mask_eroded)
area = np.sum(border)

print(area)
print(new_area)

In [ ]:
from skimage import transform
img = plt.imread("resized_data/example_segmentation_resized/ISIC_0001769_segmentation.png")
img_rot = transform.rotate(img, 0)
print(img.shape)
plt.imshow(img, cmap="gray");

In [ ]:
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

border = Image.open('resized_data/example_segmentation_resized/ISIC_0015483_segmentation.png').convert("RGB")
mask = Image.open('resized_data/example_segmentation_resized/ISIC_0015483_segmentation.png')
img = Image.open('resized_data/example_image_resized/ISIC_0015483.jpg').convert("RGB")

border.paste(img, mask=mask)

som = np.array(border)
print(som[200,300,:])

segments = slic(som, n_segments = 10, sigma = 5, mask=mask)

fig = plt.figure("Superpixels -- %d segments" % (10))
ax = fig.add_subplot(1, 1, 1)
ax.imshow(mark_boundaries(som, segments))
plt.axis("off")




#plt.show()
avg_color = [np.round(np.mean(som[segments==s], axis=0)) for s in np.unique(segments)]

avg_color
avg_color = avg_color[1:]
np.std(avg_color, axis = 0)

In [ ]:
# NOTE: Burde vi tage mean(red), E[r], fra hele billedet(inden det segmenteres, men med masken rundt om) til at regne Var(r), 
# eller kun fra vektoren med average farver, som jeg gør her? Hvad giver det mest brugbare resultat?

def computeColorSD(avg_color_array):
    """
    Computes the Standard Deviation in the individual color vectors (R, G, B) of the average color array.
    takes average color array as input.
    returns tuple(SD(r), SD(g), SD(b)).
    """
    avg_colors = np.array(avg_color_array)
    red, green, blue = avg_colors[:,0], avg_colors[:,1], avg_colors[:,2]
    mean_r, mean_g, mean_b = sum(red)/len(red), sum(green)/len(green), sum(blue)/len(blue)
    var_r = sum(red**2)/len(red) - mean_r**2
    var_g = sum(green**2)/len(green) - mean_g**2
    var_b = sum(blue**2)/len(blue) - mean_b**2
    return math.sqrt(var_r), math.sqrt(var_g), math.sqrt(var_b)

In [ ]:
def computeAvgDistance(avg_color_array):
    """
    Computes the Euclidean distance between all colors in the average color array.
    takes average color array as input.
    returns average distance as float.
    """
    avg_colors = avg_color_array
    distances = []
    for i, color in enumerate(avg_colors):
        for j in range(i+1, len(avg_colors)):
            R_1, R_2 = avg_colors[i][0], avg_colors[j][0]
            G_1, G_2 = avg_colors[i][1], avg_colors[j][1]
            B_1, B_2 = avg_colors[i][2], avg_colors[j][2]
            distance = math.sqrt((R_1 - R_2)**2+(G_1 - G_2)**2+(B_1 - B_2)**2)
            distances.append(distance)
    return sum(distances)/len(distances) # Could also return range(min, max) if we want to use that to say something.

In [ ]:
print("Testing average color and SD(color) for two very different pictures of skin/lesions.\n")
print("Even skin color:")
print("Average color distance", round(computeAvgDistance(avg_color2)))
print("Standard deviation in color (R,G,B) {}".format(computeColorSD(avg_color2)))
print()
print("Colorful lesion:")
print("Average color distance", round(computeAvgDistance(avg_color)))
print("Standard deviation in color (R,G,B) {}".format(computeColorSD(avg_color)))

In [ ]:
border2 = Image.open('resized_data/example_segmentation_resized/ISIC_0014616_segmentation.png').convert("RGB")
mask2 = Image.open('resized_data/example_segmentation_resized/ISIC_0014616_segmentation.png')
img2 = Image.open('resized_data/example_image_resized/ISIC_0014616.jpg').convert("RGB")

border2.paste(img2, mask=mask2)
#border.show()


som2 = np.array(border2)


segments2 = slic(som2, n_segments = 10, sigma = 5, mask=mask2)

fig = plt.figure("Superpixels -- %d segments" % (10))
ax = fig.add_subplot(1, 1, 1)
ax.imshow(mark_boundaries(som2, segments2))
plt.axis("off")




#plt.show()
avg_color2 = [np.round(np.mean(som2[segments2==s], axis=0)) for s in np.unique(segments2)]
avg_color2
np.std(avg_color2, axis = 0)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6840a14b-d41a-45b9-abd0-892354334cb2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>